In [ ]:
import numpy as np
import heapq
import matplotlib.pyplot as plt
import rasterio
import math

# Haversine distance
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in kilometers
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    
    a = np.sin(delta_phi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    return R * c  # Distance in kilometers

# Convert latitude and longitude to map indices
def latlon_to_index(lat, lon, lat_min, lon_min, lat_res, lon_res):
    x = int((lat - lat_min) / lat_res)
    y = int((lon - lon_min) / lon_res)
    return (4050 - x, y)

def index_to_latlon(x, y, lat_min, lon_min, lat_res, lon_res):
    lat = lat_min + (4050 - x) * lat_res
    lon = lon_min + y * lon_res
    return (lat, lon)

# Calculate effective speed based on wind
def effective_speed(ship_speed, wind_speed, wind_angle, dx, dy):
    relative_angle = math.atan2(dy, dx) - wind_angle
    wind_effect = wind_speed * math.cos(relative_angle)
    return max(0, ship_speed + wind_effect)

# Check if a move is valid
def valid_move(x, y, binary_map):
    return 0 <= x < binary_map.shape[0] and 0 <= y < binary_map.shape[1] and binary_map[x, y] == 0

# A* Algorithm with wind effects
def a_star(start, goal, binary_map, wind_speed_map, wind_angle_map, ship_speed):
    open_list = []
    heapq.heappush(open_list, (0, 0, start))
    came_from = {}
    g_score = {start: 0}
    f_score = {start: 0}

    while open_list:
        _, current_g, current = heapq.heappop(open_list)

        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            path.reverse()
            return path, g_score[goal]

        neighbors = [
            (current[0] - 1, current[1]), (current[0] + 1, current[1]),
            (current[0], current[1] - 1), (current[0], current[1] + 1)
        ]

        for neighbor in neighbors:
            if valid_move(neighbor[0], neighbor[1], binary_map):
                lat1, lon1 = index_to_latlon(current[0], current[1], lat_min, lon_min, lat_res, lon_res)
                lat2, lon2 = index_to_latlon(neighbor[0], neighbor[1], lat_min, lon_min, lat_res, lon_res)

                distance = haversine(lat1, lon1, lat2, lon2)
                wind_speed = wind_speed_map[neighbor[0], neighbor[1]]
                wind_angle = wind_angle_map[neighbor[0], neighbor[1]]
                dx, dy = neighbor[0] - current[0], neighbor[1] - current[1]
                eff_speed = effective_speed(ship_speed, wind_speed, wind_angle, dx, dy)
                time_cost = distance / eff_speed if eff_speed > 0 else float('inf')

                tentative_g_score = current_g + time_cost

                if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = tentative_g_score
                    heapq.heappush(open_list, (f_score[neighbor], tentative_g_score, neighbor))

    return None, float('inf')

# Generate random wind speed and angle matrices
def generate_wind_map(grid_shape):
    wind_speed = np.random.uniform(0, 20, grid_shape)
    wind_angle = np.random.uniform(0, 2 * np.pi, grid_shape)
    return wind_speed, wind_angle

# Load binary map
binary_file = "indian_ocean_binary.tif"
with rasterio.open(binary_file) as src:
    binary_map = src.read(1)

# Define map bounds
lat_min, lat_max = -60, 30
lon_min, lon_max = 20, 120
lat_res = (lat_max - lat_min) / binary_map.shape[0]
lon_res = (lon_max - lon_min) / binary_map.shape[1]

# Generate wind data
wind_speed_map, wind_angle_map = generate_wind_map(binary_map.shape)

# Visualize wind speed map
plt.figure(figsize=(10, 8))
plt.imshow(wind_speed_map, cmap='cool', origin='upper')
plt.colorbar(label='Wind Speed (units)')
plt.title("Wind Speed Map")
plt.xlabel("Longitude Index")
plt.ylabel("Latitude Index")
plt.grid()
plt.show()

# Define start and goal points
start = latlon_to_index(18.5, 72.5, lat_min, lon_min, lat_res, lon_res)  # Mumbai
goal = latlon_to_index(11, 90, lat_min, lon_min, lat_res, lon_res)   # Antarctica region

# Ship's constant speed in km/h
ship_speed = 50

# Run A* algorithm
path, total_time = a_star(start, goal, binary_map, wind_speed_map, wind_angle_map, ship_speed)

# Convert path indices to latitude and longitude
lat_lon_path = [index_to_latlon(x, y, lat_min, lon_min, lat_res, lon_res) for x, y in path] if path else None

# Plot the map and path
plt.imshow(binary_map, cmap='gray', origin='upper')
if path:
    path_x, path_y = zip(*path)
    plt.plot(path_y, path_x, color='red', linewidth=2, label='Path')
else:
    print("No path found.")
plt.scatter([start[1], goal[1]], [start[0], goal[0]], c=['green', 'blue'], label='Start/Goal')
plt.legend()
plt.title("A* with Wind Effects")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid()
plt.show()

if path:
    print(f"Total travel time: {total_time:.2f} hours")
else:
    print("No path could be found.")
